In [15]:
import os
import pandas as pd
import yaml
import copy
import random
from pathlib import Path
import matplotlib.pyplot as plt
import aot

In [16]:
base_dir = Path(aot.__path__[0])
core_expt_yaml_path = base_dir / "experiment/core_exp_settings.yml"
stimuli_temp_path = base_dir / "experiment/stimuli_settings_temp.yml"
core_settings = yaml.load(open(core_expt_yaml_path), Loader=yaml.FullLoader)
stimuli_settings_temp = yaml.load(open(stimuli_temp_path), Loader=yaml.FullLoader)  

settings_root_path = base_dir / core_settings["paths"]["settings_path"] 


In [17]:
subject_number = core_settings['various']['subject_number']
session_number = core_settings['various']['session_number']
run_number = core_settings['various']['run_number']
blank_number_per_run = core_settings['various']['blank_number_per_run']#only for main run
total_video_number = core_settings['various']['total_video_number']
unique_video_number_per_session = core_settings['various']['unique_video_number_per_session']
video_number_per_run_main = int((unique_video_number_per_session*4) / run_number)


video_names = []
for i in range(1, 1+total_video_number):       
    #0 padding for video name
    video_name = str(i).zfill(4)+".mp4"
    video_names.append(video_name)  

print("videos number: ", len(video_names))
origin_videos_number = len(video_names)

print("videos names: ", video_names)
print("subject number: ", subject_number)
print("session number: ", session_number)
print("run number: ", run_number)
print("unique video number per session: ", unique_video_number_per_session)



videos number:  2200
videos names:  ['0001.mp4', '0002.mp4', '0003.mp4', '0004.mp4', '0005.mp4', '0006.mp4', '0007.mp4', '0008.mp4', '0009.mp4', '0010.mp4', '0011.mp4', '0012.mp4', '0013.mp4', '0014.mp4', '0015.mp4', '0016.mp4', '0017.mp4', '0018.mp4', '0019.mp4', '0020.mp4', '0021.mp4', '0022.mp4', '0023.mp4', '0024.mp4', '0025.mp4', '0026.mp4', '0027.mp4', '0028.mp4', '0029.mp4', '0030.mp4', '0031.mp4', '0032.mp4', '0033.mp4', '0034.mp4', '0035.mp4', '0036.mp4', '0037.mp4', '0038.mp4', '0039.mp4', '0040.mp4', '0041.mp4', '0042.mp4', '0043.mp4', '0044.mp4', '0045.mp4', '0046.mp4', '0047.mp4', '0048.mp4', '0049.mp4', '0050.mp4', '0051.mp4', '0052.mp4', '0053.mp4', '0054.mp4', '0055.mp4', '0056.mp4', '0057.mp4', '0058.mp4', '0059.mp4', '0060.mp4', '0061.mp4', '0062.mp4', '0063.mp4', '0064.mp4', '0065.mp4', '0066.mp4', '0067.mp4', '0068.mp4', '0069.mp4', '0070.mp4', '0071.mp4', '0072.mp4', '0073.mp4', '0074.mp4', '0075.mp4', '0076.mp4', '0077.mp4', '0078.mp4', '0079.mp4', '0080.mp4', '00

In [18]:
def make_settings_for_one_session(
    subject_number, session_number, original_video_names, exp_tupe="main"
):  # input videos that before selection
    """
    original_video_names here should also be the uniqe original video names that will only be used in one session.
    the selection work should be done before this function (in the make_settings_for_one_subject function)
    """

    def make_settings_for_one_run(
        subject_number, session_number, run_number, video_names, exp_type="main"
    ):  # input videos that after selection
        """
        video_names: the video names that will be used directly used in the experiment,
        already selected, resampled, reversed, and doubled for a run
        this fuction just add some blanks into the video_names and save settings for one run
        """
        # i is run number
        settings = copy.deepcopy(stimuli_settings_temp)
        settings["stimuli"]["movie_files"] = video_names
        # uniformly distribute the blanks into movies and add some perturbations
        movie_number_per_blank = int(
            len(settings["stimuli"]["movie_files"]) / blank_number_per_run
        )
        for i in range(blank_number_per_run):
            perturb = random.randint(-1, 1)
            settings["stimuli"]["movie_files"].insert(
                (i + 1) * movie_number_per_blank + i + perturb, "blank"
            )

        # save settings
        with open(
            str(settings_root_path)
            + "/"
            + exp_type
            + "/experiment_settings_"
            + "sub_"
            + subject_number.zfill(2)
            + "_ses_"
            + session_number.zfill(2)
            + "_run_"
            + run_number.zfill(2)
            + ".yml",
            "w",
        ) as outfile:
            print(
                str(settings_root_path)
                + "/"
                + exp_type
                + "/experiment_settings_"
                + "sub_"
                + subject_number.zfill(2)
                + "_ses_"
                + session_number.zfill(2)
                + "_run_"
                + run_number.zfill(2)
                + ".yml",
                "w",
            )
            print(settings)
            print(" ")
            yaml.dump(settings, outfile, default_flow_style=False)
        return settings

    original_video_names = copy.deepcopy(original_video_names)

    # now the original_video_names are the names that will only be used in one session, don't need to be sub-sampled for each session

    forward_video_names = [
        name.replace(".mp4","_fw.mp4") for name in original_video_names
    ]  
    reversed_video_names = [
        name.replace(".mp4","_rev.mp4") for name in original_video_names 
    ]  
    total_video_names = copy.deepcopy(forward_video_names) + copy.deepcopy(
        reversed_video_names
    )
    doubled_total_video_names = copy.deepcopy(total_video_names) + copy.deepcopy(
        total_video_names
    )  # for main exp
    random.shuffle(total_video_names)
    random.shuffle(doubled_total_video_names)

    for i in range(1, run_number + 1):
        if i == run_number:
            # make settings for main run
            video_names = doubled_total_video_names[
                (i - 1) * video_number_per_run_main :
            ]
            make_settings_for_one_run(
                subject_number, session_number, str(i), video_names, exp_type="main"
            )
        else:
            # make settings for main run
            video_names = doubled_total_video_names[
                (i - 1) * video_number_per_run_main : i * video_number_per_run_main
            ]
            make_settings_for_one_run(
                subject_number, session_number, str(i), video_names, exp_type="main"
            )

In [19]:
def make_settings_for_one_subject(subject_number,original_video_names,exp_tupe = "main"):
    original_video_names = copy.deepcopy(original_video_names)
    #first shuffle the original video names for each subject
    random.shuffle(original_video_names)
    for i in range(1,session_number+1):
        #get i th {unique_video_number_per_session} videos for each session
        selected_video_names = original_video_names[(i-1)*unique_video_number_per_session:i*unique_video_number_per_session]
        make_settings_for_one_session(subject_number,str(i),selected_video_names,exp_tupe = exp_tupe)

def make_settings_for_all_subjects(original_video_names,exp_tupe = "main"): 
    for i in range(1,subject_number+1):
        make_settings_for_one_subject(str(i),original_video_names,exp_tupe = exp_tupe)

In [20]:
make_settings_for_all_subjects(video_names,exp_tupe = "main")

/tank/shared/2022/arrow_of_time/arrow_of_time_exp/aot/data/experiment/settings/main/experiment_settings_sub_01_ses_01_run_01.yml w
{'stimuli': {'movie_files': ['1175_rev.mp4', '1104_rev.mp4', '1309_rev.mp4', '1655_rev.mp4', '1693_fw.mp4', 'blank', '0312_rev.mp4', '1004_fw.mp4', '0989_rev.mp4', '0660_rev.mp4', '0032_fw.mp4', '1702_fw.mp4', '1817_fw.mp4', '1736_fw.mp4', 'blank', '0387_rev.mp4', '1317_rev.mp4', '1122_rev.mp4', '1300_rev.mp4', '1315_fw.mp4', 'blank', '1849_fw.mp4', '1573_fw.mp4', '0464_fw.mp4', '2026_rev.mp4', '0699_rev.mp4', '0312_fw.mp4', '2180_fw.mp4', 'blank', '0837_fw.mp4', '1122_rev.mp4', '1775_rev.mp4', '1104_fw.mp4', '1030_fw.mp4', '1161_fw.mp4', 'blank', '1305_fw.mp4', '1634_fw.mp4', '0853_fw.mp4', '1473_fw.mp4', 'blank', '1634_fw.mp4', '0172_fw.mp4', '0172_rev.mp4', '1303_fw.mp4', '1093_rev.mp4', '0481_fw.mp4', '1603_fw.mp4', 'blank', '1719_fw.mp4', '1550_fw.mp4', '0089_rev.mp4', '2137_rev.mp4', '0003_rev.mp4', '1298_rev.mp4', 'blank', '1556_rev.mp4', '0916_fw.mp

In [21]:
'''
f, s = plt.subplots(1, 1, figsize=(16,4))
rand_YTBBs, rand_MITs, randPXLs = np.random.randint(16,136, size=20), np.random.randint(16,136, size=80), np.random.randint(16,136, size=15)
for i, rnds in enumerate([rand_YTBBs, rand_MITs, randPXLs]):
    for r in rnds:
        s.axvline(r, c=['r', 'g', 'b'][i], lw=3)
s.set_ylim([-0.1,1.1])
'''


"\nf, s = plt.subplots(1, 1, figsize=(16,4))\nrand_YTBBs, rand_MITs, randPXLs = np.random.randint(16,136, size=20), np.random.randint(16,136, size=80), np.random.randint(16,136, size=15)\nfor i, rnds in enumerate([rand_YTBBs, rand_MITs, randPXLs]):\n    for r in rnds:\n        s.axvline(r, c=['r', 'g', 'b'][i], lw=3)\ns.set_ylim([-0.1,1.1])\n"